In [1]:
import sys
# append the path of the parent directory
sys.path.append("..")

In [2]:
import math
import os

import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import pandas as pd
import datetime

import random
random.seed(7)

import orjson
import gzip

from importlib import reload
from lib import sketches, visualization_utils, encoders, ploting, pacha_sketch

reload(pacha_sketch)
from lib.pacha_sketch import ADTree, get_n_updates, MaterializedCombinations

from lib import baselines
reload(baselines)
from lib.baselines import query_df, filter_df

/home/rudi/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


# Helper Functions

In [3]:
def convert_np_types(obj):
    if isinstance(obj, dict):
        return {k: convert_np_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_np_types(v) for v in obj]
    elif isinstance(obj, (np.integer, np.floating)):
        return obj.item()
    else:
        return obj

In [4]:
def generate_comb_queries(df: pd.DataFrame, num_cols: list, cat_cols: list, materialized: MaterializedCombinations, num_queries_per_n_predicates: int = 20,
                      decay_rate: float = 0.3, range_portion:float = 0.5, queryset_name: str = None, dir_path: str = None) -> list:
    # Exponentially decaying probability for each index - smaller decay_rate means faster decay
    probs_cat = decay_rate**np.arange(len(cat_cols))
    probs_cat = probs_cat / probs_cat.sum()

    relevant_combinations = {}
    for comb in materialized.relevant_combinations:
        if len(comb) not in relevant_combinations:
            relevant_combinations[len(comb)] = [comb]
        else:
            relevant_combinations[len(comb)].append(comb)
    
    all_queries = {}
    n_dim = max(len(num_cols), len(cat_cols))
    for n in np.arange(1,n_dim + 1):
        if n > len(num_cols):
            n_num = len(num_cols)
        else:
            n_num = n
        if n > len(cat_cols):
            n_cat = len(cat_cols)
        else:
            n_cat = n
        
        if n_num in relevant_combinations:
            combinations = relevant_combinations[n_num]
        else:
            combinations = []

        queries = []
        for _ in range(num_queries_per_n_predicates):
            filter_predicates = {}
            picked_cat_cols = np.random.choice(cat_cols, size=n_cat, replace=False, p=probs_cat)
            for col in picked_cat_cols:
                val_counts = df[col].value_counts(normalize=True)
                predicate = list(np.random.choice(val_counts.index, size=1, p=val_counts.values))
                filter_predicates[col] = predicate
        
            if len(combinations) > 0:
                picked_num_cols = combinations[np.random.choice(np.arange(len(combinations)))]
            else:
                picked_num_cols = np.random.choice(num_cols, size=n_num, replace=False)
            for col in picked_num_cols:
                q_25 = int(df[col].quantile(0.25))
                q_75 = int(df[col].quantile(0.75))
                range_size = int((q_75 - q_25) * range_portion)
                start = random.randint(q_25, q_75 - range_size)
                end = start + range_size
                filter_predicates[col] = (start, end)

            all_predicates = []
            for col in df.columns:
                if col in filter_predicates:
                    all_predicates.append(filter_predicates[col])
                else:
                    all_predicates.append("*")
            queries.append(all_predicates)
    
        query_set = {
            "queryset_name": queryset_name + f"{n}",
            "num_queries": num_queries_per_n_predicates,
            "queries": queries
        }

        if dir_path is not None:
            cleaned_query_set = convert_np_types(query_set)
            file_path = os.path.join(dir_path, f"{query_set['queryset_name']}.json")
            with open(file_path, 'wb') as f:
                f.write(orjson.dumps(cleaned_query_set))
                
        all_queries[n] = query_set
                
    return all_queries


In [5]:
def generate_cat_queries(df: pd.DataFrame, cat_cols: list, num_queries_per_n_predicates: int = 20,
                      decay_rate: float = 0.3, queryset_name: str = None, dir_path: str = None) -> list:
    # Exponentially decaying probability for each index - smaller decay_rate means faster decay
    probs_cat = decay_rate**np.arange(len(cat_cols))
    probs_cat = probs_cat / probs_cat.sum()
    
    all_queries = {}
    for n_cat in np.arange(1,len(cat_cols) + 1):
        queries = []
        for _ in range(num_queries_per_n_predicates):
            filter_predicates = {}
            picked_cat_cols = np.random.choice(cat_cols, size=n_cat, replace=False, p=probs_cat)
            for col in picked_cat_cols:
                val_counts = df[col].value_counts(normalize=True)
                predicate = list(np.random.choice(val_counts.index, size=1, p=val_counts.values))
                filter_predicates[col] = predicate
        
            all_predicates = []
            for col in df.columns:
                if col in filter_predicates:
                    all_predicates.append(filter_predicates[col])
                else:
                    all_predicates.append("*")
            queries.append(all_predicates)
    
        query_set = {
            "queryset_name": queryset_name + f"{n_cat}",
            "num_queries": num_queries_per_n_predicates,
            "queries": queries
        }

        if dir_path is not None:
            cleaned_query_set = convert_np_types(query_set)
            file_path = os.path.join(dir_path, f"{query_set['queryset_name']}.json")
            with open(file_path, 'wb') as f:
                f.write(orjson.dumps(cleaned_query_set))
                
        all_queries[n_cat] = query_set
                
    return all_queries


In [6]:
def generate_num_queries(df: pd.DataFrame, num_cols: list, materialized: MaterializedCombinations, num_queries_per_n_predicates: int = 20,
                      range_portion:float = 0.5, queryset_name: str = None, dir_path: str = None) -> list:
    relevant_combinations = {}
    for comb in materialized.relevant_combinations:
        if len(comb) not in relevant_combinations:
            relevant_combinations[len(comb)] = [comb]
        else:
            relevant_combinations[len(comb)].append(comb)
    
    all_queries = {}
    for n_num in np.arange(1,len(num_cols) + 1):
        if n_num in relevant_combinations:
            combinations = relevant_combinations[n_num]
        else:
            combinations = []
        queries = []
        for _ in range(num_queries_per_n_predicates):
            filter_predicates = {}
            if len(combinations) > 0:
                picked_num_cols = combinations[np.random.choice(np.arange(len(combinations)))]
            else:
                picked_num_cols = np.random.choice(num_cols, size=n_num, replace=False)
            for col in picked_num_cols:
                q_25 = int(df[col].quantile(0.25))
                q_75 = int(df[col].quantile(0.75))
                range_size = int((q_75 - q_25) * range_portion)
                start = random.randint(q_25, q_75 - range_size)
                end = start + range_size
                filter_predicates[col] = (start, end)
        
            all_predicates = []
            for col in df.columns:
                if col in filter_predicates:
                    all_predicates.append(filter_predicates[col])
                else:
                    all_predicates.append("*")
            queries.append(all_predicates)
    
        query_set = {
            "queryset_name": queryset_name + f"{n_num}",
            "num_queries": num_queries_per_n_predicates,
            "queries": queries
        }

        if dir_path is not None:
            cleaned_query_set = convert_np_types(query_set)
            file_path = os.path.join(dir_path, f"{query_set['queryset_name']}.json")
            with open(file_path, 'wb') as f:
                f.write(orjson.dumps(cleaned_query_set))
                
        all_queries[n_num] = query_set
                
    return all_queries


In [7]:
def generate_random_queries_mat(df: pd.DataFrame, num_cols: list, cat_cols: list, materialized: MaterializedCombinations, max_predicates: int = 4, num_queries: int = 100,
                      decay_rate: float = 0.3, range_portion:float = 0.25, dataset_name: str = None, file_path: str = None) -> list:
    # Exponentially decaying probability for each index - smaller decay_rate means faster decay
    probs_cat = np.array([decay_rate**i for i in range(len(cat_cols))])
    probs_cat = probs_cat / probs_cat.sum()

    relevant_combinations = []
    for comb in materialized.relevant_combinations:
        if len(comb) > max_predicates:
            continue
        else:
            relevant_combinations.append(comb)
    relevant_combinations.append([])
    
    queries = []
    for i in range(num_queries):
        picked_num_cols = relevant_combinations[np.random.choice(np.arange(len(relevant_combinations)))]
        n_predicates = random.randint(1, max_predicates)

        n_cat = max(n_predicates - len(picked_num_cols), 0)
        n_cat = min(n_cat, len(cat_cols))

        filter_predicates = {}
        if n_cat > 0:
            picked_cat_cols = np.random.choice(cat_cols, size=n_cat, replace=False, p=probs_cat)
            for col in picked_cat_cols:
                val_counts = df[col].value_counts(normalize=True)
                predicate = list(np.random.choice(val_counts.index, size=1, p=val_counts.values))
                filter_predicates[col] = predicate
        if len(picked_num_cols) > 0:
            for col in picked_num_cols:
                q_25 = int(df[col].quantile(0.25))
                q_75 = int(df[col].quantile(0.75))
                range_size = int((q_75 - q_25) * range_portion)
                start = random.randint(q_25, q_75 - range_size)
                end = start + range_size
                filter_predicates[col] = (start, end)

        all_predicates = []
        for col in df.columns:
            if col in filter_predicates:
                all_predicates.append(filter_predicates[col])
            else:
                all_predicates.append("*")
        queries.append(all_predicates)
    
    query_set = {
        "dataset_name": dataset_name,
        "num_queries": num_queries,
        "queries": queries
    }

    if file_path is not None:
        cleaned_query_set = convert_np_types(query_set)
        if file_path.endswith('.gz'):
            with gzip.open(file_path, "wb") as f:
                f.write(orjson.dumps(cleaned_query_set))
        elif file_path.endswith('.json'):
            with open(file_path, 'wb') as f:
                f.write(orjson.dumps(cleaned_query_set))
                
    return query_set


In [8]:
def generate_random_queries(df: pd.DataFrame, num_cols: list, cat_cols: list, max_predicates: int = 4, num_queries: int = 200,
                      decay_rate: float = 0.5, range_portion:float = 0.5, dataset_name: str = None, file_path: str = None) -> list:
    # Exponentially decaying probability for each index - smaller decay_rate means faster decay
    probs_cat = np.array([decay_rate**i for i in range(len(cat_cols))])
    probs_cat = probs_cat / probs_cat.sum()

    probs_num = np.array([decay_rate**i for i in range(len(num_cols))])
    probs_num = probs_num / probs_num.sum()
    
    queries = []
    for i in range(num_queries):
        n_predicates = random.randint(1, max_predicates)
        n_cat = min(random.randint(0, n_predicates), len(cat_cols))
        n_num = min(n_predicates - n_cat, len(num_cols))
        if n_num > 2 and n_num < len(num_cols):
            n_num = len(num_cols)  

        filter_predicates = {}
        if n_cat > len(cat_cols):
            continue
        picked_cat_cols = np.random.choice(cat_cols, size=n_cat, replace=False, p=probs_cat)
        for col in picked_cat_cols:
            val_counts = df[col].value_counts(normalize=True)
            predicate = list(np.random.choice(val_counts.index, size=1, p=val_counts.values))
            filter_predicates[col] = predicate

        if n_num > len(num_cols):
            continue
        picked_num_cols = np.random.choice(num_cols, size=n_num, replace=False, p=probs_num)
        for col in picked_num_cols:
            q_25 = int(df[col].quantile(0.25))
            q_75 = int(df[col].quantile(0.75))
            range_size = int((q_75 - q_25) * range_portion)
            start = random.randint(q_25, q_75 - range_size)
            end = start + range_size
            filter_predicates[col] = (start, end)

        all_predicates = []
        for col in df.columns:
            if col in filter_predicates:
                all_predicates.append(filter_predicates[col])
            else:
                all_predicates.append("*")
        queries.append(all_predicates)
    
    query_set = {
        "dataset_name": dataset_name,
        "num_queries": num_queries,
        "queries": queries
    }

    if file_path is not None:
        cleaned_query_set = convert_np_types(query_set)
        if file_path.endswith('.gz'):
            with gzip.open(file_path, "wb") as f:
                f.write(orjson.dumps(cleaned_query_set))
        elif file_path.endswith('.json'):
            with open(file_path, 'wb') as f:
                f.write(orjson.dumps(cleaned_query_set))
                
    return query_set


In [9]:
def generate_queries(df: pd.DataFrame, num_cols: list, cat_cols: list, num_queries: int = 200, n_cat: int = 1, n_num: int = 1,
                      decay_rate: float = 0.5, range_portion:float = 0.5, dataset_name: str = None, file_path: str = None) -> list:
    # Exponentially decaying probability for each index - smaller decay_rate means faster decay
    probs_cat = np.array([decay_rate**i for i in range(len(cat_cols))])
    probs_cat = probs_cat / probs_cat.sum()

    probs_num = np.array([decay_rate**i for i in range(len(num_cols))])
    probs_num = probs_num / probs_num.sum()

    queries = []
    for i in range(num_queries):
        filter_predicates = {}

        picked_cat_cols = np.random.choice(cat_cols, size=n_cat, replace=False, p=probs_cat)
        for col in picked_cat_cols:
            val_counts = df[col].value_counts(normalize=True)
            predicate = list(np.random.choice(val_counts.index, size=1, p=val_counts.values))
            filter_predicates[col] = predicate

        picked_num_cols = np.random.choice(num_cols, size=n_num, replace=False, p=probs_num)
        for col in picked_num_cols:
            q_25 = int(df[col].quantile(0.25))
            q_75 = int(df[col].quantile(0.75))
            range_size = int((q_75 - q_25) * range_portion)
            start = random.randint(q_25, q_75 - range_size)
            end = start + range_size
            filter_predicates[col] = (start, end)

        all_predicates = []
        for col in df.columns:
            if col in filter_predicates:
                all_predicates.append(filter_predicates[col])
            else:
                all_predicates.append("*")
        queries.append(all_predicates)
    
    query_set = {
        "dataset_name": dataset_name,
        "n_cat": n_cat,
        "n_num": n_num,
        "num_queries": num_queries,
        "range_size": range_size,
        "queries": queries
    }

    if file_path is not None:
        cleaned_query_set = convert_np_types(query_set)
        if file_path.endswith('.gz'):
            with gzip.open(file_path, "wb") as f:
                f.write(orjson.dumps(cleaned_query_set))
        elif file_path.endswith('.json'):
            with open(file_path, 'wb') as f:
                f.write(orjson.dumps(cleaned_query_set))
                
    return query_set


In [10]:
def profile_num_cols(df: pd.DataFrame, num_cols: list[str], bases: list[int] = None, range_portion: float = 0.5):
    if bases is None:
        bases = [2] * len(num_cols)
    ranges_df = (df[num_cols].quantile(0.75) - df[num_cols].quantile(0.25)) * range_portion
    print("Range sizes for numerical columns:")
    print(ranges_df)
    range_levels = np.log(ranges_df) / np.log(bases)
    print("\nApprox level for every range query:")
    print(range_levels)
    max_levels = np.log(df[num_cols].max()) / np.log(bases)
    print("\nMax levels:")
    print(max_levels)
    

In [11]:
def generate_ad_tree(df: pd.DataFrame, cat_cols: list[str], file_path:str = None) -> ADTree:
    """
    Generate an ADTree from the given DataFrame.
    
    :param df: DataFrame containing the data.
    :param cat_cols: List of categorical columns to be used in the ADTree.
    :param file_path: Optional path to save the ADTree.
    :return: An instance of ADTree.
    """
    ad_tree = ADTree()

    for col in cat_cols:
        ad_tree.add_dimension(set(df[col].unique().tolist()), name=col)

    print(ad_tree.names)
    
    if file_path is not None:
        ad_tree.save_to_file(file_path)
    return ad_tree

In [12]:
def partition_and_save(df: pd.DataFrame, n_partitions: int, base_path: str):
    dir_path = os.path.dirname(base_path)
    if dir_path and not os.path.exists(dir_path):
        os.makedirs(dir_path, exist_ok=True)
    partition_size = len(df) // n_partitions
    for i in range(n_partitions):
        start_idx = i * partition_size
        end_idx = (i + 1) * partition_size if i < n_partitions - 1 else len(df)
        partition = df.iloc[start_idx:end_idx]
        partition.to_parquet(f"{base_path}_part_{i+1}.parquet", index=False)

In [46]:
from collections import defaultdict
from typing import List, Dict, Tuple

# ---------- helper -----------------------------------------------------------
def _numeric_predicate_for_selectivity(
    series: pd.Series,
    target: float,
    tol: float = 0.01
) -> Tuple[Tuple[int, int], float]:
    """
    Return a (start, end) integer range expected to hit ~target selectivity
    and the estimated selectivity achieved.
    """
    n = len(series)
    # empirical CDF
    values = series.sort_values().values
    width = max(1, int(target * n))
    # pick random anchor
    anchor = random.randint(0, max(0, n - width))
    start = int(values[anchor])
    end = int(values[min(anchor + width - 1, n - 1)])
    # estimate selectivity
    sel = (series.between(start, end).sum()) / n
    return (start, end), sel

def _category_for_selectivity(
    series: pd.Series,
    target: float
) -> Tuple[str, float]:
    """
    Return a category value whose empirical frequency is closest to target.
    """
    freqs = series.value_counts(normalize=True)
    idx = (freqs - target).abs().idxmin()
    return idx, freqs[idx]

# ---------- main -------------------------------------------------------------
def generate_queries_by_selectivity(
    df: pd.DataFrame,
    num_cols: List[str],
    cat_cols: List[str],
    target_selectivity: float,
    n_queries: int,
    decay_rate: float = 0.5,
    tol: float = 0.01,
    max_attempts: int = 30,
    seed: int = None
) -> List[List]:
    """
    Generate `n_queries` queries whose expected selectivity is
    within ±tol of `target_selectivity`.

    Returns a list of predicate lists (same order as df.columns).
    """
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    probs_cat = decay_rate**np.arange(len(cat_cols))
    probs_cat = probs_cat / probs_cat.sum() * 0.5

    probs_num = decay_rate**np.arange(len(num_cols))
    probs_num = probs_num / probs_num.sum() * 0.5

    probs = np.concatenate((probs_cat, probs_num))

    queries = []
    full_cols = list(df.columns)

    while len(queries) < n_queries:
        remaining_cols = set(full_cols)
        predicates: Dict[str, object] = {}
        est_sel = 1.0  # start with the whole table
        attempts = 0

        while attempts < max_attempts and abs(est_sel - target_selectivity) > tol:
            attempts += 1
            # random pick a column that is still free
            col = np.random.choice(full_cols, p=probs)
            # ensure we only pick from remaining_cols
            while col not in remaining_cols:
                col = np.random.choice(full_cols, p=probs)
            remaining_cols.remove(col)

            if col in num_cols:
                pred, sel = _numeric_predicate_for_selectivity(
                    df[col], target_selectivity if est_sel == 1 else target_selectivity / est_sel, tol
                )
            else:  # categorical
                pred, sel = _category_for_selectivity(
                    df[col], target_selectivity if est_sel == 1 else target_selectivity / est_sel
                )

            predicates[col] = pred
            est_sel *= sel

            # break early if no columns left to refine
            if not remaining_cols:
                break

        # if still outside tolerance, restart
        if abs(est_sel - target_selectivity) > tol:
            continue

        # build predicate list in column order
        query = []
        for col in full_cols:
            query.append(predicates.get(col, "*"))
        queries.append(query)

    return queries


In [14]:
def generate_queries_selectivities(df: pd.DataFrame, num_cols: list[str], cat_cols: list[str], 
                                    target_selectivities: np.ndarray, n_queries: int, tol: float = 0.01, 
                                    max_attempts: int = 30, seed: int = 7, test_queries = False,
                                    query_set_name: str = None, dir_path: str= None) -> list[list]:
    all_query_sets = {}
    for sel in target_selectivities:
        qs = generate_queries_by_selectivity(
            df=df,               # your DataFrame
            num_cols=num_cols,
            cat_cols=cat_cols,
            target_selectivity=sel,
            n_queries=n_queries,                   # 20 queries per target
            tol=tol,
            seed=seed,
            max_attempts=max_attempts
        )
        print(f"Done: {sel}")
        all_query_sets[sel] = qs
    
    for i, sel in enumerate(all_query_sets.keys()):
        corrected = []
        for j, q in enumerate(all_query_sets[sel]):
            correct_q = []
            for pred in q:
                if pred != '*' and not isinstance(pred, tuple):
                    correct_q.append([pred])
                else:
                    correct_q.append(pred)
            corrected.append(correct_q)
        all_query_sets[sel] = corrected
    
    if test_queries:
        true_sel = np.empty((len(all_query_sets.keys()), n_queries), dtype=float)
        for i, sel in enumerate(all_query_sets.keys()):
            for j, q in enumerate(all_query_sets[sel]):
                res = query_df(df, q)
                true_sel[i,j] = res / len(df)
        print("True selectivities:")
        print(np.mean(true_sel, axis=1))

    if query_set_name is not None and dir_path is not None:
        for i, sel in enumerate(all_query_sets.keys()):  
            name = f"{query_set_name}_{sel}"

            query_set = {
                "queryset_name": name,
                "queries": all_query_sets[sel]
            }
            file_path = os.path.join(dir_path, f"{query_set['queryset_name']}.json")
            if file_path is not None:
                cleaned_query_set = convert_np_types(query_set)
                with open(file_path, 'wb') as f:
                    f.write(orjson.dumps(cleaned_query_set))

    return all_query_sets

In [15]:
def test_selectivities(df, queries):
    sel = np.empty(len(queries), dtype=float)
    for i, query in enumerate(queries):
        res = query_df(df, query)
        sel[i] = res / len(df)
    sel = pd.DataFrame(sel, columns=['selection'])
    print(sel.describe())
    return sel

# Bank Marketing

https://archive.ics.uci.edu/dataset/222/bank+marketing

## Data Cleaning

In [ ]:
df = pd.read_parquet("raw/bank_marketing/bank_full.parquet")

In [ ]:
# Convert month and day to day of the year indexed from 0
df['date'] = df.apply(lambda row: (datetime.datetime.strptime(f"{row['month']} {row['day']}", "%b %d").timetuple().tm_yday - 1), axis=1)

In [ ]:
clean_df = df[['poutcome', 'job', 'education', 'housing', 'loan', 'marital', 'duration', 'balance', 'age', 'date']]

In [32]:
clean_df.to_parquet("clean/bank_marketing.parquet", index=False)

## Partition Dataset

In [ ]:
bank_df = pd.read_parquet("clean/bank_marketing.parquet")

partition_and_save(bank_df, 10, "partitioned/bank_marketing_10/bank_marketing")

## Queries and AD-Tree Generation

In [56]:
bank_df = pd.read_parquet("clean/bank_marketing.parquet")
cat_cols = ['poutcome', 'job', 'education', 'housing', 'loan', 'marital']
num_cols = ['duration', 'balance', 'age', 'date']

column_names = cat_cols + num_cols

relevant_combinations = [
    ['duration'],
    ['balance'],
    ['age'],
    ['date'],
    
    ['duration', 'age'],
    ['duration', 'date'],
    ['duration', 'balance'],
    ['balance', 'age'],
    ['balance', 'date'],
    
    ['duration', 'balance', 'age'],
    
    ['duration', 'balance', 'age', 'date']
    ]

mat_combinations = MaterializedCombinations(col_names=num_cols, relevant_combinations=relevant_combinations)


In [33]:
len(relevant_combinations)

11

In [34]:
bank_df.nunique()

poutcome        4
job            12
education       4
housing         2
loan            2
marital         3
duration     1573
balance      7168
age            77
date          318
dtype: int64

In [6]:
bank_df[num_cols].describe()

,duration,balance,age,date
count,45211.000000,45211.000000,45211.000000,45211.000000
mean,258.163080,1362.272058,40.936210,170.198668
std,257.527812,3044.765829,10.618762,74.597784
min,0.000000,-8019.000000,18.000000,5.000000
25%,103.000000,72.000000,33.000000,128.000000
50%,180.000000,448.000000,39.000000,154.000000
75%,319.000000,1428.000000,48.000000,216.000000
max,4918.000000,102127.000000,95.000000,364.000000


In [27]:
profile_num_cols(bank_df, num_cols, bases=[4, 5, 2, 2])

Range sizes for numerical columns:
duration    108.0
balance     678.0
age           7.5
date         44.0
dtype: float64

Approx level for every range query:
duration    3.377444
balance     4.050574
age         2.906891
date        5.459432
dtype: float64

Max levels:
duration    6.131928
balance     7.166460
age         6.569856
date        8.507795
dtype: float64


In [35]:
ad_tree = ADTree()

for col in cat_cols:
    ad_tree.add_dimension(set(bank_df[col].unique().tolist()), name=col)
ad_tree.collapse_last_dimension()
ad_tree.save_to_file("../sketches/ad_trees/bank_marketing.json")


True

In [53]:
rand_queries = generate_random_queries_mat(df=bank_df, num_cols=num_cols, cat_cols=cat_cols, materialized=mat_combinations, range_portion=0.5, max_predicates=4, num_queries=200, dataset_name="bank_random", file_path="../queries/bank/bank_random.json")

In [54]:
sel = np.empty(len(rand_queries['queries']), dtype=float)
for i, query in enumerate(rand_queries['queries']):
    res = query_df(bank_df, query)
    sel[i] = res / len(bank_df)
    # print(f"{i}: {res} -> {res/sel[i]}")
sel = pd.DataFrame(sel, columns=['selection'])
sel.describe()

,selection
count,200.000000
mean,0.059982
std,0.100568
min,0.000022
25%,0.007471
50%,0.031983
75%,0.063679
max,0.817478


In [39]:
_

{np.float64(0.01): [['*',
   [np.int64(3)],
   '*',
   '*',
   '*',
   '*',
   '*',
   '*',
   '*',
   '*'],
  [[np.int64(1)], '*', '*', '*', '*', '*', '*', '*', '*', (198, 216)],
  ['*', '*', '*', '*', [np.int64(312)], '*', '*', '*', '*', '*'],
  ['*', '*', [np.int64(4)], '*', '*', '*', '*', '*', '*', '*'],
  ['*', '*', '*', [np.int64(4)], '*', '*', '*', '*', '*', '*'],
  ['*', '*', [np.int64(4)], '*', '*', '*', '*', '*', '*', '*'],
  ['*', [np.int64(3)], '*', '*', '*', '*', '*', '*', '*', '*'],
  ['*', '*', '*', [np.int64(1)], '*', '*', '*', (7, 8), '*', '*'],
  ['*', '*', '*', '*', '*', [np.int64(9)], '*', '*', '*', '*'],
  ['*', '*', '*', '*', [np.int64(312)], '*', '*', '*', '*', '*'],
  ['*', '*', [np.int64(4)], '*', '*', '*', '*', '*', '*', '*'],
  ['*', '*', '*', '*', '*', '*', [np.int64(4610)], '*', '*', '*'],
  ['*', '*', '*', '*', '*', '*', [np.int64(4610)], '*', '*', '*'],
  ['*', '*', [np.int64(4)], '*', '*', '*', '*', '*', '*', '*'],
  ['*', '*', '*', '*', '*', [np.int64(9

In [57]:
target_selectivities = np.array([0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64])
_ = generate_queries_selectivities(df=bank_df, num_cols=num_cols, cat_cols=cat_cols, 
                                    target_selectivities=target_selectivities, n_queries=200, 
                                    test_queries=True, query_set_name="bank_sel", dir_path="../queries/bank/selectivities")

Done: 0.01
Done: 0.02
Done: 0.04
Done: 0.08
Done: 0.16
Done: 0.32
Done: 0.64
True selectivities:
[0.00980503 0.02084836 0.04025801 0.08112583 0.16174416 0.32199432
 0.64135741]


In [57]:
all_cat_queries = generate_cat_queries(df=bank_df, cat_cols=cat_cols, num_queries_per_n_predicates=200, queryset_name="bank_cat_", dir_path="../queries/bank/categorical")

In [56]:
all_num_queries = generate_num_queries(df=bank_df, num_cols=num_cols, materialized=mat_combinations, num_queries_per_n_predicates=200, queryset_name="bank_num_", dir_path="../queries/bank/numerical")

In [62]:
all_comb_queries = generate_comb_queries(df=bank_df, cat_cols=cat_cols, num_cols=num_cols, materialized=mat_combinations, num_queries_per_n_predicates=200, queryset_name="bank_mix_", dir_path="../queries/bank/mixed")

In [64]:
idx = 1
sel = np.empty(len(all_comb_queries[idx]['queries']), dtype=float)
for i, query in enumerate(all_comb_queries[idx]['queries']):
    res = query_df(bank_df, query)
    sel[i] = res / len(bank_df)
    # print(f"{i}: {res} -> {res/sel[i]}")
sel = pd.DataFrame(sel, columns=['selection'])
sel.describe()

,selection
count,200.000000
mean,0.067674
std,0.048157
min,0.000796
25%,0.023988
50%,0.063104
75%,0.106252
max,0.187499


### Old

In [ ]:
generate_queries(df=bank_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=200, n_cat=1, n_num=1, dataset_name="bank_marketing_2_cols", file_path="../queries/bank_marketing_2_cols.json")

In [ ]:
generate_queries(df=bank_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=200, n_cat=2, n_num=2, dataset_name="bank_marketing_4_cols", file_path="../queries/bank_marketing_4_cols.json")

In [ ]:
generate_random_queries(df=bank_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=500, dataset_name="bank_random", file_path="../queries/bank/bank_random.json")

# Online Retail

https://archive.ics.uci.edu/dataset/352/online+retail

In [15]:
df = pd.read_parquet("raw/online_retail/online_retail.parquet")

In [ ]:
df['category'] = df['StockCode'].str.slice(0, 3)
df['date'] = pd.to_datetime(df['InvoiceDate'], format="%d.%m.%Y %H:%M").dt.strftime('%d.%m.%Y').rank(method='dense').astype(int) - 1

In [ ]:
df_customer = pd.DataFrame(df['CustomerID'].unique(), columns=['CustomerID'])
df_customer['age'] = generate_bounded_normal(loc=35, scale=10, size=len(df_customer), low=18, high=76)
df_customer['gender'] = np.random.choice(['m', 'f', 'd'], size=len(df_customer), replace=True, p=[0.48, 0.49, 0.03])
df_merged = df.merge(df_customer, on='CustomerID', how='inner')
df_merged["region"] = df_merged['Country']
df_merged['total'] = df_merged['Quantity'] * df_merged['UnitPrice']
df_merged['total'] = df_merged['total'].abs().fillna(0).astype(int)

In [ ]:
clean_df = df_merged[['category', 'region', 'gender', 'date', 'age', 'total']].copy()

In [ ]:
clean_df.to_parquet("clean/online_retail.parquet", index=False)

## Partition Dataset

In [ ]:
retail_df = pd.read_parquet("clean/online_retail.parquet")

partition_and_save(retail_df, 10, "partitioned/online_retail_10/online_retail")

## Queries and AD-Tree Generation

In [53]:
retail_df = pd.read_parquet("clean/online_retail.parquet")
cat_cols = ['category', 'region', 'gender']
num_cols = ['date', 'age', 'total']

column_names = cat_cols + num_cols

relevant_combinations = [
    ['date'],
    ['age'],
    ['total'],
    
    ['date', 'age'],
    ['date', 'total'],
    ['age', 'total'],
    
    ['date', 'age', 'total']
    ]

mat_combinations = MaterializedCombinations(col_names=num_cols, relevant_combinations=relevant_combinations)


In [17]:
len(retail_df)

541909

In [34]:
retail_df.nunique()

category     125
region        38
gender         3
total       1088
age           51
date         305
dtype: int64

In [61]:
retail_df[num_cols].describe()

,date,age,total
count,541909.000000,541909.000000,541909.000000
mean,151.535738,35.104582,20.881216
std,88.132067,7.753374,378.638866
min,0.000000,18.000000,0.000000
25%,74.000000,31.000000,3.000000
50%,150.000000,34.000000,9.000000
75%,228.000000,39.000000,17.000000
max,304.000000,71.000000,168469.000000


In [31]:
profile_num_cols(retail_df, num_cols, bases=[4, 2, 2])

Range sizes for numerical columns:
date     77.0
age       4.0
total     7.0
dtype: float64

Approx level for every range query:
date     3.133393
age      2.000000
total    2.807355
dtype: float64

Max levels:
date      4.123964
age       6.149747
total    17.362124
dtype: float64


In [51]:
generate_ad_tree(df=retail_df, cat_cols=cat_cols, file_path="sketches/ad_trees/online_retail.json")

['category', 'region', 'gender']


In [78]:
rand_queries = generate_random_queries_mat(df=retail_df, num_cols=num_cols, cat_cols=cat_cols, materialized=mat_combinations, range_portion=0.5, max_predicates=4, num_queries=200, dataset_name="retail_random", file_path="../queries/retail/retail_random.json")

In [79]:
test_selectivities(retail_df, rand_queries['queries'])

        selection
count  200.000000
mean     0.059440
std      0.107346
min      0.000004
25%      0.002902
50%      0.015290
75%      0.063838
max      0.914320


,selection
0,0.000642
1,0.005158
2,0.001716
3,0.062021
4,0.000712
...,...
195,0.020839
196,0.020991
197,0.252122
198,0.041450


In [54]:
target_selectivities = np.array([0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64])
_ = generate_queries_selectivities(df=retail_df, num_cols=num_cols, cat_cols=cat_cols, 
                                    target_selectivities=target_selectivities, n_queries=200, 
                                    test_queries=True, query_set_name="retail_sel", dir_path="../queries/retail/selectivities")

Done: 0.01
Done: 0.02
Done: 0.04
Done: 0.08
Done: 0.16
Done: 0.32
Done: 0.64
True selectivities:
[0.0121984  0.0204097  0.04165291 0.08281876 0.16341581 0.32294128
 0.64388582]


In [80]:
all_cat_queries = generate_cat_queries(df=retail_df, cat_cols=cat_cols, num_queries_per_n_predicates=200, queryset_name="retail_cat_", dir_path="../queries/bank/categorical")

In [81]:
all_num_queries = generate_num_queries(df=retail_df, num_cols=num_cols, materialized=mat_combinations, num_queries_per_n_predicates=200, queryset_name="retail_num_", dir_path="../queries/retail/numerical")

In [82]:
all_comb_queries = generate_comb_queries(df=retail_df, cat_cols=cat_cols, num_cols=num_cols, materialized=mat_combinations, num_queries_per_n_predicates=200, queryset_name="retail_mix_", dir_path="../queries/retail/mixed")

### Old

In [ ]:
generate_queries(df=retail_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=200, n_cat=1, n_num=1, dataset_name="online_retail_2_cols", file_path="queries/online_retail_2_cols.json")

In [ ]:
generate_queries(df=retail_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=200, n_cat=2, n_num=2, dataset_name="online_retail_4_cols", file_path="queries/online_retail_4_cols.json")

In [ ]:
generate_random_queries(df=retail_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=500, dataset_name="online_retail_random", file_path="../queries/retail/online_retail_random.json")

# Folktables

https://github.com/socialfoundations/folktables

| Column  | Description                                                                       |
| ------- | --------------------------------------------------------------------------------- |
| `SEX`   | **Sex** — 1 for male, 2 for female.                                               |
| `RAC1P` | **Race** — Detailed race code (White, Black, Asian, etc.).                        |
| `SCHL`  | **Educational attainment** — Highest degree or level of school completed.         |
| `MAR`   | **Marital status** — E.g., married, divorced, widowed, never married.             |
| `POBP`  | **Place of birth** — Numeric code for U.S. state or foreign country of birth.     |
| `COW`   | **Class of worker** — Employment type (e.g., private, government, self-employed). |
| `OCCP`  | **Occupation code** — Detailed job classification (4-digit code).                 |
| `AGEP`  | **Age of person** — Age in years (0–99, top-coded at 99).                         |
| `PINCP` | **Total person income** — Total pre-tax income in the past 12 months.             |
| `PWGTP` | **Person’s weight** — Statistical weight used to produce population estimates.    |



In [ ]:
from folktables import ACSDataSource, ACSIncome, generate_categories, adult_filter

In [ ]:
data_source = ACSDataSource(survey_year='2017', horizon='1-Year', survey='person')
ca_data = data_source.get_data(states=["CA"], download=False)


In [54]:
cat_cols = ["SEX", "RAC1P", "SCHL", "MAR", "POBP", "COW", "OCCP"]
num_cols = ["AGEP", "PINCP", "PWGTP"]

In [ ]:
folk_df = ca_data[cat_cols + num_cols]
folk_df.fillna(0, inplace=True)
folk_df = folk_df.astype(int)

In [ ]:
folk_df.to_parquet("clean/acs_folktables.parquet", index=False)

## Partition Dataset

In [ ]:
folk_df = pd.read_parquet("clean/acs_folktables.parquet")

partition_and_save(folk_df, 10, "partitioned/acs_folktables_10/acs_folktables")

## Queries and AD-Tree Generation

In [49]:
census_df = pd.read_parquet("clean/acs_folktables.parquet")
cat_cols = ["SEX", "RAC1P", "SCHL", "MAR", "POBP", "COW", "OCCP"]
num_cols = ["AGEP", "PINCP", "PWGTP"]

column_names = cat_cols + num_cols

relevant_combinations = [
    ['AGEP'],
    ['PINCP'],
    ['PWGTP'],
    
    ['AGEP', 'PINCP'],
    ['AGEP', 'PWGTP'],
    ['PINCP', 'PWGTP'],
    
    ['AGEP', 'PINCP', 'PWGTP']
    ]

mat_combinations = MaterializedCombinations(col_names=num_cols, relevant_combinations=relevant_combinations)


In [88]:
folk_df.nunique()

SEX         2
RAC1P       9
SCHL       25
MAR         5
POBP      223
COW        10
OCCP      480
AGEP       92
PINCP    7877
PWGTP     811
dtype: int64

In [42]:
res = pd.read_csv("../results/census/selectivities/census_sel_0.01.csv")

In [44]:
census_ad_tree = ADTree.from_json("../sketches/ad_trees/acs_folktables.json")

In [45]:
census_ad_tree.compute_distinct_values()

np.int64(2408400000)

In [89]:
folk_df[num_cols].describe()

,AGEP,PINCP,PWGTP
count,377575.000000,3.775750e+05,377575.000000
mean,39.979792,3.736253e+04,104.712052
std,23.141297,6.580092e+04,75.371747
min,0.000000,-7.000000e+03,1.000000
25%,20.000000,0.000000e+00,59.000000
50%,39.000000,1.500000e+04,81.000000
75%,58.000000,4.800000e+04,126.000000
max,94.000000,1.259000e+06,1494.000000


In [38]:
profile_num_cols(census_df, num_cols, bases=[2, 10, 2])

Range sizes for numerical columns:
AGEP        19.0
PINCP    24000.0
PWGTP       33.5
dtype: float64

Approx level for every range query:
AGEP     4.247928
PINCP    4.380211
PWGTP    5.066089
dtype: float64

Max levels:
AGEP      6.554589
PINCP     6.100026
PWGTP    10.544964
dtype: float64


In [92]:
generate_ad_tree(df=folk_df, cat_cols=cat_cols, file_path="sketches/ad_trees/acs_folktables.json")

['SEX', 'RAC1P', 'SCHL', 'MAR', 'POBP', 'COW', 'OCCP']


In [17]:
rand_queries = generate_random_queries_mat(df=census_df, num_cols=num_cols, cat_cols=cat_cols, materialized=mat_combinations, range_portion=0.5, max_predicates=4, num_queries=200, dataset_name="census_random", file_path="../queries/census/census_random.json")

In [ ]:
target_selectivities = np.array([0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64])
_ = generate_queries_selectivities(df=census_df, num_cols=num_cols, cat_cols=cat_cols, 
                                    target_selectivities=target_selectivities, n_queries=200, 
                                    test_queries=True, query_set_name="census_sel", dir_path="../queries/census/selectivities")

Done: 0.01
Done: 0.02
Done: 0.04
Done: 0.08
Done: 0.16
Done: 0.32
Done: 0.64
True selectivities:
[0.01124373 0.02205551 0.04315814 0.0806909  0.16132355 0.32393856
 0.6430178 ]


In [20]:
all_cat_queries = generate_cat_queries(df=census_df, cat_cols=cat_cols, num_queries_per_n_predicates=200, queryset_name="census_cat_", dir_path="../queries/census/categorical")

In [21]:
all_num_queries = generate_num_queries(df=census_df, num_cols=num_cols, materialized=mat_combinations, num_queries_per_n_predicates=200, queryset_name="census_num_", dir_path="../queries/census/numerical")

In [22]:
all_comb_queries = generate_comb_queries(df=census_df, cat_cols=cat_cols, num_cols=num_cols, materialized=mat_combinations, num_queries_per_n_predicates=200, queryset_name="census_mix_", dir_path="../queries/census/mixed")

### Old

In [8]:
generate_queries(df=folk_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=200, n_cat=1, n_num=1, dataset_name="acs_folktables_2_cols", file_path="queries/acs_folktables_2_cols.json")

{'dataset_name': 'acs_folktables_2_cols',
 'n_cat': 1,
 'n_num': 1,
 'num_queries': 200,
 'range_size': 19,
 'queries': [[[np.int64(2)], '*', '*', '*', '*', '*', '*', (30, 49), '*', '*'],
  [[np.int64(1)], '*', '*', '*', '*', '*', '*', (24, 43), '*', '*'],
  ['*', '*', '*', '*', [np.int64(6)], '*', '*', (32, 51), '*', '*'],
  [[np.int64(1)], '*', '*', '*', '*', '*', '*', '*', '*', (62, 95)],
  [[np.int64(2)], '*', '*', '*', '*', '*', '*', (22, 41), '*', '*'],
  ['*', [np.int64(1)], '*', '*', '*', '*', '*', (37, 56), '*', '*'],
  [[np.int64(1)], '*', '*', '*', '*', '*', '*', '*', '*', (65, 98)],
  [[np.int64(1)], '*', '*', '*', '*', '*', '*', (31, 50), '*', '*'],
  ['*', [np.int64(2)], '*', '*', '*', '*', '*', '*', (19096, 43096), '*'],
  [[np.int64(1)], '*', '*', '*', '*', '*', '*', (21, 40), '*', '*'],
  ['*', [np.int64(1)], '*', '*', '*', '*', '*', '*', (16627, 40627), '*'],
  [[np.int64(2)], '*', '*', '*', '*', '*', '*', '*', (7035, 31035), '*'],
  ['*', [np.int64(1)], '*', '*', '*'

In [9]:
generate_queries(df=folk_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=200, n_cat=2, n_num=2, dataset_name="acs_folktables_4_cols", file_path="queries/acs_folktables_4_cols.json")

{'dataset_name': 'acs_folktables_4_cols',
 'n_cat': 2,
 'n_num': 2,
 'num_queries': 200,
 'range_size': 19,
 'queries': [[[np.int64(2)],
   [np.int64(1)],
   '*',
   '*',
   '*',
   '*',
   '*',
   (32, 51),
   '*',
   (68, 101)],
  [[np.int64(2)],
   [np.int64(1)],
   '*',
   '*',
   '*',
   '*',
   '*',
   '*',
   (11383, 35383),
   (75, 108)],
  [[np.int64(1)],
   [np.int64(8)],
   '*',
   '*',
   '*',
   '*',
   '*',
   (31, 50),
   (19735, 43735),
   '*'],
  ['*',
   [np.int64(9)],
   [np.int64(0)],
   '*',
   '*',
   '*',
   '*',
   (23, 42),
   (15536, 39536),
   '*'],
  ['*',
   '*',
   [np.int64(16)],
   '*',
   '*',
   '*',
   [np.int64(9130)],
   (35, 54),
   (3779, 27779),
   '*'],
  [[np.int64(1)],
   '*',
   [np.int64(22)],
   '*',
   '*',
   '*',
   '*',
   (34, 53),
   '*',
   (89, 122)],
  [[np.int64(2)],
   [np.int64(1)],
   '*',
   '*',
   '*',
   '*',
   '*',
   (35, 54),
   (10218, 34218),
   '*'],
  [[np.int64(2)],
   [np.int64(3)],
   '*',
   '*',
   '*',
   '*',

In [37]:
generate_random_queries(df=folk_df, num_cols=num_cols, cat_cols=cat_cols, num_queries=500, dataset_name="census_random", file_path="../queries/census/census_random.json")

{'dataset_name': 'census_random',
 'num_queries': 500,
 'queries': [[[np.int64(2)],
   '*',
   [np.int64(6)],
   '*',
   '*',
   '*',
   '*',
   '*',
   '*',
   '*'],
  [[np.int64(2)],
   [np.int64(9)],
   [np.int64(19)],
   '*',
   '*',
   '*',
   '*',
   (26, 45),
   '*',
   '*'],
  [[np.int64(1)],
   '*',
   '*',
   [np.int64(3)],
   '*',
   '*',
   [np.int64(430)],
   '*',
   '*',
   '*'],
  ['*', '*', '*', '*', '*', '*', '*', '*', (10220, 34220), '*'],
  [[np.int64(1)], '*', [np.int64(0)], '*', '*', '*', '*', '*', '*', '*'],
  ['*',
   [np.int64(1)],
   '*',
   '*',
   '*',
   '*',
   '*',
   (36, 55),
   (4112, 28112),
   (64, 97)],
  ['*', [np.int64(6)], [np.int64(16)], '*', '*', '*', '*', '*', '*', '*'],
  [[np.int64(2)], '*', '*', '*', '*', '*', '*', '*', '*', '*'],
  ['*',
   '*',
   '*',
   [np.int64(1)],
   '*',
   '*',
   '*',
   (39, 58),
   (16307, 40307),
   (64, 97)],
  ['*', '*', '*', '*', '*', '*', '*', (38, 57), (995, 24995), (70, 103)],
  [[np.int64(1)],
   [np.int

# TPCH

In [58]:
column_names = [
    "c_shipmode", "c_returnflag", "c_linestatus",
    "c_discount", "c_tax",
    "n_shipdate", "n_commitdate", "n_receiptdate",
    "n_extendedprice", "n_quantity"
]

cat_cols = ['c_shipmode', 'c_returnflag', 'c_linestatus', 'c_discount', 'c_tax']
num_cols = ['n_shipdate', 'n_commitdate', 'n_receiptdate', 
            'n_extendedprice', 'n_quantity']

relevant_combinations = [
    ['n_shipdate'],
    ['n_commitdate'],
    ['n_receiptdate'],
    ['n_extendedprice'],
    ['n_quantity'],
    
    ['n_shipdate', 'n_commitdate'],
    ['n_shipdate', 'n_receiptdate'],
    ['n_shipdate', 'n_quantity'],
    ['n_commitdate', 'n_receiptdate'],
    ['n_commitdate', 'n_extendedprice'],
    ['n_extendedprice', 'n_quantity'],
    
    ['n_commitdate', 'n_receiptdate', 'n_extendedprice'],
    
    ['n_shipdate', 'n_commitdate', 'n_receiptdate', 'n_extendedprice', 'n_quantity']
    ]

mat_combinations = MaterializedCombinations(col_names=num_cols, relevant_combinations=relevant_combinations)


In [60]:
lineitem_0_1= pd.read_parquet("../data/tpch/lineitem_0.1.parquet")
lineitem_0_1[num_cols].describe()

,n_shipdate,n_commitdate,n_receiptdate,n_extendedprice,n_quantity
count,599934.000000,599934.000000,599934.000000,599934.000000,599934.000000
mean,1264.133661,1263.104053,1279.636327,37022.099154,25.533409
std,695.868904,695.140572,695.927287,22577.271692,14.420349
min,1.000000,29.000000,2.000000,902.000000,1.000000
25%,665.000000,665.000000,681.000000,18151.000000,13.000000
50%,1264.000000,1264.000000,1280.000000,35502.000000,26.000000
75%,1865.000000,1864.000000,1881.000000,53246.000000,38.000000
max,2525.000000,2494.000000,2551.000000,99950.000000,50.000000


In [56]:
lineitem_0_5[['n_shipdate', 'n_commitdate', 'n_receiptdate', 'n_extendedprice', 'n_quantity']].describe()

,n_shipdate,n_commitdate,n_receiptdate,n_extendedprice,n_quantity
count,2999671.00,2999671.00,2999671.00,2999671.00,2999671.00
mean,1263.87,1262.89,1279.37,36979.98,25.51
std,695.22,694.53,695.28,22573.24,14.42
min,0.00,29.00,2.00,901.00,1.00
25%,664.00,663.00,679.00,18108.00,13.00
50%,1264.00,1263.00,1279.00,35448.00,26.00
75%,1864.00,1863.00,1880.00,53218.00,38.00
max,2525.00,2494.00,2555.00,99950.00,50.00


In [261]:
np.prod(lineitem_0_5[num_cols].max()-lineitem_0_5[num_cols].min())

np.int64(81045520482668000)

In [ ]:
ad_tree = ADTree()
for col in cat_cols:
    ad_tree.add_dimension(set(lineitem_8[col].unique().tolist()), name=col)
ad_tree.collapse_last_dimension()
ad_tree.save_to_file("../sketches/ad_trees/tpch_lineitem.json")

In [255]:
generate_random_queries_mat(df=lineitem_0_5, num_cols=num_cols, cat_cols=cat_cols, materialized=mat_combinations, range_portion=0.5, max_predicates=3, num_queries=100, dataset_name="tpch_random", file_path="../queries/tpch/tpch_random_2.json")

{'dataset_name': 'tpch_random',
 'num_queries': 100,
 'queries': [[['REG AIR'],
   ['N'],
   '*',
   '*',
   '*',
   '*',
   '*',
   (1055, 1655),
   '*',
   '*'],
  [['MAIL'], '*', '*', '*', '*', '*', '*', (1194, 1794), '*', '*'],
  [['MAIL'], '*', '*', '*', '*', (847, 1447), '*', '*', '*', '*'],
  ['*', '*', '*', '*', '*', (667, 1267), (1164, 1764), '*', '*', '*'],
  ['*', '*', '*', '*', '*', '*', (903, 1503), (1136, 1736), '*', '*'],
  [['MAIL'], ['N'], ['O'], '*', '*', '*', '*', '*', '*', '*'],
  ['*', '*', '*', '*', '*', '*', (846, 1446), (1163, 1763), '*', '*'],
  ['*', '*', '*', '*', '*', '*', '*', '*', (21616, 39171), (14, 26)],
  ['*', '*', '*', '*', '*', '*', '*', '*', (29857, 47412), '*'],
  ['*', '*', '*', '*', [np.float64(0.02)], '*', '*', '*', '*', (18, 30)],
  ['*', '*', '*', '*', '*', '*', '*', '*', (32590, 50145), (21, 33)],
  [['FOB'], '*', '*', '*', '*', (705, 1305), '*', '*', '*', (13, 25)],
  [['SHIP'], ['N'], '*', '*', '*', (797, 1397), '*', '*', '*', '*'],
  ['*'

In [250]:
sel = np.empty(len(tpch_queries['queries']), dtype=float)
for i, query in enumerate(tpch_queries['queries']):
    res = query_df(lineitem_0_5, query)
    sel[i] = res / len(lineitem_0_5)
    # print(f"{i}: {res} -> {res/sel[i]}")
sel = pd.DataFrame(sel, columns=['selection'])

In [251]:
sel.describe()

,selection
count,100.00
mean,0.10
std,0.09
min,0.00
25%,0.03
50%,0.06
75%,0.15
max,0.50


In [61]:
target_selectivities = np.array([0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64])
_ = generate_queries_selectivities(df=lineitem_0_1, num_cols=num_cols, cat_cols=cat_cols, 
                                    target_selectivities=target_selectivities, n_queries=200, 
                                    test_queries=True, query_set_name="tpch_sel", dir_path="../queries/tpch/selectivities")

Done: 0.01
Done: 0.02
Done: 0.04
Done: 0.08
Done: 0.16
Done: 0.32
Done: 0.64
True selectivities:
[0.01094494 0.01992794 0.03835107 0.08077427 0.16312654 0.32051644
 0.64038342]


In [17]:
targets = np.array([0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64])
# targets = np.array([0.064])

all_query_sets = {}
for sel in targets:
    qs = generate_queries_by_selectivity(
        df=lineitem_0_1,               # your DataFrame
        num_cols=['n_shipdate','n_commitdate','n_receiptdate',
                  'n_extendedprice','n_quantity'],
        cat_cols=['c_shipmode','c_returnflag','c_linestatus',
                  'c_discount','c_tax'],
        target_selectivity=sel,
        n_queries=200,                   # 20 queries per target
        tol=0.01,
        seed=39
    )
    print(f"Done: {sel}")
    all_query_sets[sel] = qs

for i, sel in enumerate(all_query_sets.keys()):
    corrected = []
    for j, q in enumerate(all_query_sets[sel]):
        correct_q = []
        for pred in q:
            if pred != '*' and not isinstance(pred, tuple):
                correct_q.append([pred])
            else:
                correct_q.append(pred)
        corrected.append(correct_q)
    all_query_sets[sel] = corrected

Done: 0.01
Done: 0.02
Done: 0.04
Done: 0.08
Done: 0.16
Done: 0.32
Done: 0.64


In [19]:
true_sel = np.empty((len(all_query_sets.keys()), 200))
for i, sel in enumerate(all_query_sets.keys()):
    for j, q in enumerate(all_query_sets[sel]):
        res = query_df(lineitem_0_1, q)
        true_sel[i,j] = res / len(lineitem_0_1)

In [20]:
np.mean(true_sel, axis=1)

array([0.01130613, 0.02032711, 0.04112887, 0.08073912, 0.16413017,
       0.32250026, 0.64081326])

In [21]:
for i, sel in enumerate(all_query_sets.keys()):  
    name = f"tpch_sel_{sel}"

    query_set = {
        "dataset_name": name,
        "queries": all_query_sets[sel]
    }
    file_path = f"../queries/tpch/selectivities/tpch_sel_{sel}.json" 
    if file_path is not None:
        cleaned_query_set = convert_np_types(query_set)
        with open(file_path, 'wb') as f:
            f.write(orjson.dumps(cleaned_query_set))

In [29]:
all_cat_queries = generate_cat_queries(df=lineitem_0_1, cat_cols=cat_cols, num_queries_per_n_predicates=200, queryset_name="tpch_cat_", dir_path="../queries/tpch/categorical")

In [68]:
all_num_queries = generate_num_queries(df=lineitem_0_1, num_cols=num_cols, materialized=mat_combinations, num_queries_per_n_predicates=200, queryset_name="tpch_num_", dir_path="../queries/tpch/numerical")

In [69]:
all_comb_queries = generate_comb_queries(df=lineitem_0_1, cat_cols=cat_cols, num_cols=num_cols, materialized=mat_combinations, num_queries_per_n_predicates=200, queryset_name="tpch_mix_", dir_path="../queries/tpch/mixed")

# OLD

### Self-Generated Datasets

In [ ]:
def generate_normal_dataset(size, num_columns, mean, std_dev, output_path=None):
    """
    Generates a dataset with the specified parameters where each column
    contains integer values following a normal distribution.

    Parameters:
    - size (int): Number of rows in the dataset.
    - num_columns (int): Number of columns in the dataset.
    - mean (float): Mean of the normal distribution.
    - std_dev (float): Standard deviation of the normal distribution.
    - output_dir (str, optional): Directory to save the dataset in Parquet format. Defaults to None.

    Returns:
    - pd.DataFrame: Generated dataset as a pandas DataFrame.
    """
    # Generate random data for each column
    data = {
        f"d_{i}": np.random.normal(loc=mean, scale=std_dev, size=size).astype(int)
        for i in range(num_columns)
    }

    # Create DataFrame
    df = pd.DataFrame(data)

    # Save to output directory if specified
    if output_path:
        df.to_parquet(output_path, index=False)
        print(f"Dataset saved to {output_path}")

    return df


In [ ]:
df = generate_normal_dataset(size=100000, num_columns=3, mean=5000, std_dev=50, output_path="normal_3d_100k.parquet")

In [ ]:
df = pd.read_parquet("normal_3d_100k.parquet")

In [ ]:
df.describe()

## 6 Dimensions Dataset

In [ ]:
def generate_bounded_normal(loc, scale, size, low, high):
    result = []
    while len(result) < size:
        samples = np.random.normal(loc=loc, scale=scale, size=size)
        valid_samples = samples[(samples >= low) & (samples <= high)]
        result.extend(valid_samples.astype(int))
    return np.array(result[:size])

In [ ]:
def generate_fake_dataset(size, output_path=None):

    regions = [
        "Baden-Württemberg", "Bavaria", "Berlin", "Brandenburg", "Bremen", 
        "Hamburg", "Hesse", "Lower Saxony", "Mecklenburg-Vorpommern", 
        "North Rhine-Westphalia", "Rhineland-Palatinate", "Saarland", 
        "Saxony", "Saxony-Anhalt", "Schleswig-Holstein", "Thuringia"
    ]
    gender = ["m","f","d"]
    product_category = [chr(i) for i in range(97, 123)]

    p_product_category = np.random.zipf(1.5, len(product_category))
    p_product_category = p_product_category / np.sum(p_product_category)

    reference_dist = pd.read_parquet("reference_dist.parquet")

    ages_array = generate_bounded_normal(loc=35, scale=10, size=size, low=18, high=76)

    data = {
        "region": np.random.choice(regions, size=size, replace=True),
        "gender": np.random.choice(gender, size=size, replace=True, p=[0.48, 0.49, 03]),
        "category": np.random.choice(product_category, size=size, replace=True, p=p_product_category),
        "date": np.random.choice(reference_dist['date'], size=size, replace=True),
        "total": np.random.choice(reference_dist['total'], size=size, replace=True),
        "age": ages_array      
    }

    # Create DataFrame
    df = pd.DataFrame(data)

    # Save to output directory if specified
    if output_path:
        df.to_parquet(output_path, index=False)
        print(f"Dataset saved to {output_path}")

    return df


In [ ]:
df = generate_fake_dataset(size=200000, output_path="paper_example_200k.parquet")

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['age'], bins=30, kde=True, color='blue')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Generate random ages with a normal distribution
ages_array = np.random.normal(loc=35, scale=10, size=1000).astype(int)

# Clip the ages to ensure they fall within the range of 18 to 76
ages_array = np.clip(ages_array, 18, 76)

## Kaggle

## E-Commerce Sales Dataset

In [ ]:
df = pd.read_csv("raw/amazon_sale_report.csv", low_memory=False)
df.to_parquet("raw/amazon_sale_report.parquet", index=False)

In [ ]:
df = pd.read_parquet("raw/amazon_sale_report.parquet")

In [ ]:
df.columns

In [ ]:
df_column_subset = df[['Date', 'Status',
       'Style', 'Category', 'Size',
       'Qty', 'Amount',
       'ship-state']].copy()

In [ ]:
df_column_subset['total'] = df_column_subset['Amount'].fillna(0).astype(int)

In [ ]:
df_column_subset['date'] = pd.to_datetime(df_column_subset['Date'], format='%m-%d-%y').rank(method='dense').astype(int) - 1

In [ ]:
df_column_subset[['date', 'total']].to_parquet("raw/reference_dist.parquet", index=False)

In [ ]:
df_column_subset.nunique()